In [1]:
import numpy as np
from sklearn.decomposition import FastICA
from sklearn.decomposition import PCA
from sklearn.decomposition import KernelPCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from matplotlib import image
import glob
from sklearn.model_selection import train_test_split
import cv2

In [25]:
# load image as pixel array
#data = image.imread('metal_moderate_001_new.jpg')

directoryGlass = 'C:\\Users\\Nida\Desktop\\Lectures\\Pattern\\Final\\FMD\\image\\glass\\'
directoryMetal = 'C:\\Users\\Nida\Desktop\\Lectures\\Pattern\\Final\\FMD\\image\\metal\\'

generalDataX = []
generalDataY = [] # 0 for glass, 1 for metal


for filename in glob.glob(directoryGlass+'*.jpg'):
    generalDataX.append(np.array(cv2.imread(filename)))
    generalDataY.append(0)#glass
for filename in glob.glob(directoryGlass+'*.jpg'):
    generalDataX.append(np.array(cv2.imread(filename)))
    generalDataY.append(1)#metal
numImages = np.array(generalDataX).shape[0]
generalDataXFlattened = np.array([generalDataX[i].flatten() for i in range(0,numImages)])

print(len(generalDataX[0][0]), len(generalDataX[0][:,0]), len(generalDataX[0][0][0]))
print(len(generalDataX))
X_train, X_test, y_train, y_test = train_test_split(generalDataXFlattened, generalDataY, test_size=0.33, random_state=42)
print(len(X_train), len(X_test), len(y_train), len(y_test))

512 384 3
200
134 66 134 66
